In [23]:
import pandas as pd
from sqlalchemy import create_engine
import duckdb


# Specify the file path for the DuckDB database
db_path = '/Users/MacUser/hedonism-wines_app/database.db'  # Example path, replace with your desired path

# Establish a connection to an in-memory DuckDB database
conn = duckdb.connect(database=db_path, read_only=False)


In [30]:
results = conn.execute("""                
                        SELECT *
                          FROM whisky_stocks_table
                       WHERE import_date = '2024-03-28'
                       AND code not in (
                        SELECT code 
                          FROM whisky_stocks_table
                       WHERE import_date = '2024-03-27')
            """).fetchdf()
	# Convert the results to a DataFrame
df = pd.DataFrame(results)
df

,code,title,size,style,country,type,availability,price_gbp,import_date,url
0,HED60746,Balvenie Tun 1509 Batch 3,70cl,Spirits,Scotland,Whisky,3.0,685.000000,2024-03-28,https://hedonism.co.uk/product/balvenie-tun-15...
1,HED61722,Glenfarclas 40 Year Old 1976,70cl,Spirits,Scotland,Whisky,1.0,1850.000000,2024-03-28,https://hedonism.co.uk/product/glenfarclas-40-...
2,HED45207,Glen Elgin 7 Year Old Piper Chapter 7 Fable 2014,70cl,Spirits,Scotland,Whisky,1.0,68.699997,2024-03-28,https://hedonism.co.uk/product/glen-elgin-7-ye...
3,HED45723,Rampur Double Cask Single Malt,70cl,Spirits,India,Whisky,3.0,58.700001,2024-03-28,https://hedonism.co.uk/product/rampur-double-c...
4,HED46404,Macallan Speymalt Gordon & Macphail (Bottled 2...,70cl,Spirits,Scotland,Whisky,1.0,560.000000,2024-03-28,https://hedonism.co.uk/product/macallan-speyma...
5,HED21175,Glenlivet 25 Year Old The Sample Room,70cl,Spirits,Scotland,Whisky,6.0,415.000000,2024-03-28,https://hedonism.co.uk/product/glenlivet-25-ye...
6,HED21644,Bowmore 18 Year Old Aston Martin Edition 3,70cl,Spirits,Scotland,Whisky,1.0,275.000000,2024-03-28,https://hedonism.co.uk/product/bowmore-18-year...
7,HED00070,Balvenie Tun 1858 Batch 1,70cl,Spirits,Scotland,Whisky,1.0,1695.000000,2024-03-28,https://hedonism.co.uk/product/balvenie-tun-18...
8,HED00072,Ardbeg 8 Year Old Single Cote Rotie Cask 2323 ...,70cl,Spirits,Scotland,Whisky,1.0,1500.000000,2024-03-28,https://hedonism.co.uk/product/ardbeg-8-year-o...
9,HED00078,Macallan 20 Year Old Speymalt Gordon & MacPhai...,70cl,Spirits,Scotland,Whisky,1.0,595.000000,2024-03-28,https://hedonism.co.uk/product/macallan-20-yea...


In [35]:
results = conn.execute("""                
                       SELECT SUM (stocks_count) / 2 
                       FROM (
                        SELECT code, COUNT (*) stocks_count  
                          FROM whisky_stocks_table
                       WHERE import_date = '2024-03-25'
                       GROUP BY code
                       HAVING count (*) > 1
                       ORDER BY 2 desc)
            """).fetchdf()
	# Convert the results to a DataFrame
df = pd.DataFrame(results)
df

,(sum(stocks_count) / 2)
0,NaN


: 

In [2]:
results = conn.execute("""                

                WITH x AS (
                        SELECT COUNT (*) duplicate_count, code	  
                          FROM whisky_stocks_table
                        WHERE import_date = '2024-03-28'
                        GROUP BY code
                        HAVING COUNT (*) > 1
                )
                       SELECT s.code, s.title, s.size, s.style, s.country, s.type, 
                       round (avg (s.availability),0) availability, 
                       s.price_gbp, s.import_date, s.url
                       FROM stocks_table s JOIN x ON s.code = x.code
                       WHERE s.import_date = '2024-03-28'
                       GROUP BY s.code, s.title, s.size, s.style, s.country, s.type, s.price_gbp, s.import_date, s.url,
                       ORDER BY 1 DESC
            """).fetchdf()
	# Convert the results to a DataFrame
df = pd.DataFrame(results)
df.head()

,code,title,size,style,country,type,availability,price_gbp,import_date,url
0,HED93971,Macallan 18 Year Old Sherry Oak (2023 Release),70cl,Spirits,Scotland,Whisky,54.0,395.000000,2024-03-28,https://hedonism.co.uk/product/macallan-18-yea...
1,HED90397,Dalmore Vintage 2007,70cl,Spirits,Scotland,Whisky,4.0,120.000000,2024-03-28,https://hedonism.co.uk/product/dalmore-vintage...
2,HED8932,Kavalan Single Malt,70cl,Spirits,"Taiwan, Province of China",Whisky,3.0,71.699997,2024-03-28,https://hedonism.co.uk/product/kavalan-single-...
3,HED89075,Kavalan Solist Vinho Barrique 56.3%,70cl,Spirits,"Taiwan, Province of China",Whisky,3.0,215.000000,2024-03-28,https://hedonism.co.uk/product/kavalan-solist-...
4,HED86657,Yamazaki Distillers Reserve,70cl,Spirits,Japan,Whisky,17.0,76.500000,2024-03-28,https://hedonism.co.uk/product/yamazaki-distil...


In [34]:
df.to_csv('/Users/MacUser/hedonism-wines_app/correct_rows_2024_03_28.csv',index=False)

In [13]:
# Execute DELETE queries and commit the changes
delete_query_1 = f"""DELETE FROM 
                    stocks_table 
                    WHERE import_date = '2024-03-28'
                    AND code IN (
                        SELECT DISTINCT code FROM (
                                    SELECT COUNT (*) duplicate_count, code	  
                                            FROM whisky_stocks_table
                                            WHERE import_date = '2024-03-28'
                                            GROUP BY code
                                            HAVING COUNT (*) > 1))"""
conn.execute(delete_query_1)

In [14]:
conn.commit()

In [16]:
df=pd.read_csv('/Users/MacUser/hedonism-wines_app/correct_rows_2024_03_28.csv')
df.head()

,code,title,size,style,country,type,availability,price_gbp,import_date,url
0,HED93971,Macallan 18 Year Old Sherry Oak (2023 Release),70cl,Spirits,Scotland,Whisky,54.0,395.0,2024-03-28,https://hedonism.co.uk/product/macallan-18-yea...
1,HED90397,Dalmore Vintage 2007,70cl,Spirits,Scotland,Whisky,4.0,120.0,2024-03-28,https://hedonism.co.uk/product/dalmore-vintage...
2,HED8932,Kavalan Single Malt,70cl,Spirits,"Taiwan, Province of China",Whisky,3.0,71.7,2024-03-28,https://hedonism.co.uk/product/kavalan-single-...
3,HED89075,Kavalan Solist Vinho Barrique 56.3%,70cl,Spirits,"Taiwan, Province of China",Whisky,3.0,215.0,2024-03-28,https://hedonism.co.uk/product/kavalan-solist-...
4,HED86657,Yamazaki Distillers Reserve,70cl,Spirits,Japan,Whisky,17.0,76.5,2024-03-28,https://hedonism.co.uk/product/yamazaki-distil...


In [17]:
    # Get the column names from the DataFrame
columns = list(df.columns)

    # Generate the list of column names for the INSERT INTO statement
column_names = ", ".join(columns)

    # Generate the list of parameter placeholders (?, ?, ?) for the VALUES clause
parameter_placeholders = ", ".join(["?" for _ in range(len(columns))])

    # Convert the DataFrame to records list
records = df.values.tolist()

    # Define the name of your existing table
table_name = 'stocks_table'

    # Construct the SQL INSERT INTO statement dynamically
sql_insert = f"INSERT INTO {table_name} ({column_names}) VALUES ({parameter_placeholders})"

    # Execute the INSERT statement
conn.executemany(sql_insert, records)

In [18]:
conn.commit()

In [21]:
conn.close()

In [19]:
conn.execute("""CREATE OR REPLACE VIEW whisky_stocks_table AS 
                            SELECT * FROM stocks_table 
                            WHERE type = 'Whisky'""")